In [1]:
import pandas as pd
import numpy as np

In [3]:
data= pd.read_csv('../dataset/train_balance.csv', quotechar='"', encoding='utf8',header=None)

D:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\Lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
label=data[0]
text=data[1]
text

0         @switchfoot http://twitpic.com/2y1zl - Awww, t...
1         is upset that he can't update his Facebook by ...
2         @Kenichan I dived many times for the ball. Man...
3           my whole body feels itchy and like its on fire 
4         @nationwideclass no, it's not behaving at all....
5                             @Kwesidei not the whole crew 
6                                               Need a hug 
7         @LOLTrish hey  long time no see! Yes.. Rains a...
8                      @Tatiana_K nope they didn't have it 
9                                 @twittera que me muera ? 
10              spring break in plain city... it's snowing 
11                               I just re-pierced my ears 
12        @caregiving I couldn't bear to watch it.  And ...
13        @octolinz16 It it counts, idk why I did either...
14        @smarrison i would've been the first, but i di...
15        @iamjazzyfizzle I wish I got to watch it with ...
16        Hollis' death scene will hurt 

In [5]:
import re

# Uncomment the line below if loading from previously saved CSV

# Remove punctuation


# Remove unnecessary line breaks
data['text_processed'] = text.map(lambda x: re.sub(r"\n", '', x))
# Convert the titles to lowercase
data['text_processed'] = data['text_processed'].map(lambda x: x.lower())
# Clean the url


data['text_processed'] = data['text_processed'].map(lambda x: re.sub(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})', '', x))
data['text_processed'] = data['text_processed'].map(lambda x: re.sub('https?', ' ', x))
data['text_processed'] = data['text_processed'].map(lambda x: re.sub(r'(@[A-Za-z]+)"?', '', x))
data['text_processed'] = data['text_processed'].map(lambda x: re.sub(r'(#[A-Za-z]+)"?', '', x))
data['text_processed'] = data['text_processed'].map(lambda x: re.sub('[,\\.!?]', ' ', x))



# Print out the first rows 
print(data['text_processed'].head())

# Removing duplicate tweets?
unique_tweets = list(set(list(data['text_processed'].values)))
unique_tweets = [t for t in unique_tweets if t]

print(data['text_processed'])

0      - awww  that's a bummer   you shoulda got da...
1    is upset that he can't update his facebook by ...
2     i dived many times for the ball  managed to s...
3      my whole body feels itchy and like its on fire 
4     no  it's not behaving at all  i'm mad  why am...
Name: text_processed, dtype: object
0           - awww  that's a bummer   you shoulda got da...
1         is upset that he can't update his facebook by ...
2          i dived many times for the ball  managed to s...
3           my whole body feels itchy and like its on fire 
4          no  it's not behaving at all  i'm mad  why am...
5                                       not the whole crew 
6                                               need a hug 
7          hey  long time no see  yes   rains a bit  onl...
8                              _k nope they didn't have it 
9                                           que me muera   
10              spring break in plain city    it's snowing 
11                           

In [6]:
import gensim
from gensim.utils import simple_preprocess

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Do you want to modify this by adding more stop words?
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
  return [[word for word in simple_preprocess(str(doc)) 
    if word not in stop_words] for doc in texts]


data1 = data.text_processed.values.tolist()

data_words = list(sent_to_words(data1))

# remove stop words
data_words = remove_stopwords(data_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AW\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:

index=0
wordlist = []
score_list = []
word_statistic = {}
word_statistic_count = {}
sentence_unique={}

for sentence in data_words:
    sentence_string=[]
    sentence_string=''.join(sentence)
    if sentence_string not in sentence_unique:
        sentence_unique[sentence_string] = 1
        score=label[index]/2-1
        for word in sentence:
            if word in word_statistic:
                word_statistic[word] += score
                word_statistic_count[word] += 1
            else:
                word_statistic[word] = score
                word_statistic_count[word] = 1
    index+=1
for word in word_statistic:
    if word_statistic_count[word]>=20:
        wordlist.append(word)
        score_list.append(word_statistic[word]/word_statistic_count[word]) 

In [8]:
#lexicon_list={}
#for i in range(len(wordlist)):
    #lexicon_list[wordlist[i]]=score_list[i]
my_lexicon=pd.DataFrame(wordlist,index=None)
my_lexicon[1]=score_list
my_lexicon

,0,1
0,awww,-0.251744
1,bummer,-0.837472
2,shoulda,-0.584158
3,got,-0.130261
4,david,0.197581
5,carr,0.200000
6,third,-0.054152
7,day,0.102316
8,upset,-0.856436
9,update,0.023381


In [9]:
my_lexicon.to_csv("my_lexicon.txt", index=None,header=None,sep='\t')